In [1]:
import os
import csv 
import sys
import json
import re
import pandas as pd
import numpy as np

In [7]:
import pandas as pd
import csv
def generate_prompt(question):
    prompt_template = """
    You are a physics expert tasked with answering questions.
    Answer the following question with a detailed explanation:
    Question: {question}
    provide the final answer in a boxed format like this:
    \\boxed{{Your final answer here}}
    """
    # \n\n\\boxed{Your final answer here}\n\n
    return prompt_template.format(question=question)

In [9]:
from together import Together
# load .env file
from dotenv import load_dotenv
load_dotenv()
# Set the API key
TOGETHER_API_KEY = os.getenv("together_api_key")
client = Together(api_key=TOGETHER_API_KEY)
def together_api_call(prompt):

    import ollama
    response = ollama.chat(
    model="tinyllama",
    messages=[
        {"role": "system", "content": """You are a physics expert tasked with answering questions. Answer the following question with a detailed explanation. Provide the final answer in a boxed format like this: \n\n\\boxed{Your final answer here}\n\n"""},
        {"role": "user", "content": prompt}
    ],
    )

# # Now parse the JSON safely
    output = response["message"]["content"]
    # remove everything up to and including the </think> tag
    if "</think>" in output:
        output = output.split("</think>")[-1].strip()

    print(output)
    explaination = output
    # Check if the response contains a boxed answer
    boxed_answer = re.search(r'\\boxed{(.*?)}', explaination)
    answer = None
    if boxed_answer:
        # Extract the answer from the boxed format
        answer = boxed_answer.group(1).strip()

    return explaination, answer

In [10]:
import json

def process_questions(input_json_path: str, output_json_path: str):
    """
    Reads questions from a JSON file of the form:
      [
        {
          "data": [
            {
              "unique_id": "...",
              "topics": [...],
              "concepts": [...],
              "question": "...",
              "file_conversation_log": "..."
            },
            ...
          ]
        }
      ]
    Generates an explanation and answer for each entry (using your generate_prompt +
    together_api_call), then writes out the same structure to output_json_path,
    with each entry extended to include 'explanation' and 'answer'.
    """
    # 1. Load the input JSON
    with open(input_json_path, 'r') as f:
        json_list = json.load(f)
        

    # 2. Iterate through all entries and augment
    for container in json_list:
        for entry in container.get('data', []):
            q_text = entry.get('question')
            if not q_text:
                # skip anything without a question
                continue

            prompt = generate_prompt(q_text)
            print(f"Processing question: {q_text}")
            explanation, answer = together_api_call(prompt)
            print(f"Generated explanation: {explanation}")
            print(f"Extracted answer: {answer}")
            entry['explanation'] = explanation
            entry['answer'] = answer

    # 3. Dump back out
    with open(output_json_path, 'w') as f:
        json.dump(json_list, f, indent=2, ensure_ascii=False)






# Example usage:
processed = process_questions('final_questions_physics.json','output_with_answers.json')


Processing question: Consider a three-dimensional topological superconductor described by a Bogoliubov–de Gennes Hamiltonian with pointlike uranium impurity scattering of strength \( U \). The clean system exhibits an \( A_u \) fully gapped pairing symmetry analogous to the B phase of superfluid \({}^3\mathrm{He}\), with gap amplitude \(\Delta_0\). Within the self-consistent Born approximation and assuming dilute impurities, derive the impurity-averaged quasiparticle density of states \( N(E) \) at energy \( E \). Express \( N(E) \) symbolically in terms of \( U \), \(\Delta_0\), the normal-state density of states \( N_0 \) at the Fermi level, and energy \( E \).
Sure, I can provide you with the answer to the question based on your given instructions. Answer:

Question: Consider a three-dimensional topological superconductor described by a Bogoliubov–de Gennes Hamiltonian with pointlike uraniuum impurity scattering of strength $U$. The clean system exhibits an $A_u$ fully gapped pairin